## API call Functions
This section handles all API calls to octopus

In [ ]:

import requests
import json



# Your Octopus Energy API key


# API_KEY = "YOUR_API_KEY"



# Function to fetch electricity consumption data for a specific MPAN and meter serial number



def get_electricity_consumption(mpan, meter_serial_number, period_from, period_to):


    url = f"https://api.octopus.energy/v1/electricity-meter-points/{mpan}/meters/{






        meter_serial_number}/consumption?page_size=14000&period_from={period_from}&period_to={period_to}"

    response = requests.get(url, auth=(API_KEY, ''))

    return response.json()



# Function to fetch tariff prices for a specific product and tariff code



def get_tariff_prices_old(tariff_code, period_from, period_to):
    product_code = tariff_code[5:-2]

    url = f"https://api.octopus.energy/v1/products/{product_code}/electricity-tariffs/{






        tariff_code}/standard-unit-rates?page_size=14000&period_from={period_from}&period_to={period_to}"

    # print(url)
    response = requests.get(url, auth=(API_KEY, ''))

    return response.json()


def get_tariff_prices(tariff_code, period_from, period_to):
    product_code = tariff_code[5:-2]
    url = f"https://api.octopus.energy/v1/products/{product_code}/electricity-tariffs/{
        tariff_code}/standard-unit-rates?page_size=14000&period_from={period_from}&period_to={period_to}"
    response = requests.get(url, auth=(API_KEY, ''))
    data = response.json()
    all_data = data['results']

    while data['next'] is not None:
        # Get the next page URL
        next_page_url = data['next']
        response = requests.get(next_page_url, auth=(API_KEY, ''))
        data = response.json()
        all_data.extend(data['results'])

    return all_data


# Main function to gather consumption data and tariff costs



def get_electricity_data(account_number, start_date, end_date):


    # Get account details to get MPAN and meter information


    account_url = f"https://api.octopus.energy/v1/accounts/{account_number}/"

    account_response = requests.get(account_url, auth=(API_KEY, ''))

    account_data = account_response.json()


    # Extract MPAN and meter information

    mpans = [meter_point['mpan']






             for meter_point in account_data['properties'][0]['electricity_meter_points']]

    meter_serial_numbers = [meter['serial_number']






                            for meter in account_data['properties'][0]['electricity_meter_points'][0]['meters']]
    tariff_code = account_data['properties'][0]['electricity_meter_points'][0]['agreements'][0]['tariff_code']


    # Fetch consumption data for each MPAN and meter

    consumption_data = {}
    mpan_file = open('mpan_meter.csv', 'w')

    for mpan in mpans:

        for meter_serial_number in meter_serial_numbers:

            mpan_file.write(f'{mpan}-{meter_serial_number}\n')

            consumption_data[(mpan, meter_serial_number)] = get_electricity_consumption(






                mpan, meter_serial_number, start_date, end_date)
            with open(f'{mpan}-{meter_serial_number}.json', 'w') as f:
                json.dump(consumption_data[(mpan, meter_serial_number)], f)

    mpan_file.close()

    # Fetch tariff prices for all tariffs associated with the account
    tariff_file = open('tariffs.csv', 'w')

    tariff_codes = [tariff['tariff_code'] for tariff in account_data['properties']






                    [0]['electricity_meter_points'][0]['agreements']]

    tariff_prices = {}

    for tariff_code in tariff_codes:
        tariff_file.write(f'{tariff_code}\n')

        tariff_prices[tariff_code] = get_tariff_prices(






            tariff_code, start_date, end_date)
        with open(f'{tariff_code}.json', 'w') as f:
            json.dump(tariff_prices[tariff_code], f)

    tariff_file.close()

    return {






        "consumption_data": consumption_data,






        "tariff_prices": tariff_prices






    }

## API calling execution
This part is the actual execution to get tarrifs and consumption based on functions above

In [4]:

# Usage example
account_number = "A-3FAEDC7D"
start_date = "2024-04-19T01:00:00Z"
end_date = "2025-01-27T00:00:00Z"

# load keys

# get API key from file
with open('apikey.txt', 'r') as file:
    API_KEY = file.read()

electricity_data = get_electricity_data(account_number, start_date, end_date)

## Download energy selling price in Agile

In [35]:
# Get export cost data
export_data = get_tariff_prices(
    'E-1R-AGILE-OUTGOING-BB-23-02-28-H', start_date, end_date)
with open('export.json', 'w') as f:
    json.dump(export_data, f)

## Run this section after each time the data gets downloaded to convert Jsons to CSV

In [ ]:

import json
import csv
import os


def json_to_csv(json_file_path, csv_file_path):
    """
    Converts a JSON file to a CSV file.

    Args:
        json_file_path: Path to the JSON file.
        csv_file_path: Path to the output CSV file.
    """

    try:
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        results = data.get('results', [])  # Extract the 'results' list

        if not results:
            print("No 'results' found in the JSON file.")
            return

        # Extract field names from the first dictionary in results
        fieldnames = results[0].keys() if results else []

        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()  # Write the header row
            writer.writerows(results)  # Write the data rows

        print(f"Successfully converted {json_file_path} to {csv_file_path}")

    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {json_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Get the current directory
current_dir = os.getcwd()

for filename in os.listdir(current_dir):
    # Check if the file is a JSON file
    if filename.endswith(".json"):

        # Construct the full file path
        json_file_name = filename
        # Define the output CSV file name based on the JSON file name
        csv_file_name = json_file_name.replace(".json", ".csv")
        json_to_csv(json_file_name, csv_file_name)

An error occurred: 0
An error occurred: 0
An error occurred: 0
Successfully converted export.json to export.csv


# breakpoint All files downloaded now. 
## Rest of this code is junk. Develop code to merge 3 CSV files - consumption, Buy rate, sell rate


In [12]:
import pandas as pd
import os


def merge_csv_files(tariffs_file="tariffs.csv"):
    """
    Merges CSV files listed in a tariffs file, adding a column indicating the source file.

    Args:
        tariffs_file (str, optional): Path to the tariffs CSV file. Defaults to "tariffs.csv".

    Returns:
        pandas.DataFrame: Merged DataFrame with a 'source_file' column, or empty DataFrame on error.
    """
    try:
        tariff_files_df = pd.read_csv(
            tariffs_file, header=None, names=['filename'])
        all_data = []

        for filename in tariff_files_df['filename']:
            filepath = f"{filename}.csv"
            if os.path.exists(filepath):
                try:
                    df = pd.read_csv(filepath, parse_dates=True)
                    # Add the source filename as a new column
                    df['source_file'] = filename
                    all_data.append(df)
                except pd.errors.EmptyDataError:
                    print(f"Warning: File {filepath} is empty.")
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")
            else:
                print(f"Warning: File {filepath} not found.")

        if all_data:
            merged_df = pd.concat(all_data, ignore_index=True)
            return merged_df
        else:
            print("No CSV files could be read. Returning empty DataFrame.")
            return pd.DataFrame()

    except FileNotFoundError:
        print(f"Error: Tariffs file '{tariffs_file}' not found.")
        return pd.DataFrame()
    except pd.errors.EmptyDataError:
        print(f"Error: Tariffs file '{tariffs_file}' is empty.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()


# Example usage:
merged_tariffs = merge_csv_files('tariffs.csv')
merged_kWh = merge_csv_files('mpan_meter.csv')

In [19]:
# merged_tariffs.head()
merged_kWh.head()

,consumption,interval_start,interval_end,source_file
0,0.287,2025-01-27T00:00:00Z,2025-01-27T00:30:00Z,2600002565617-22L4294637
1,0.296,2025-01-26T23:30:00Z,2025-01-27T00:00:00Z,2600002565617-22L4294637
2,0.445,2025-01-26T23:00:00Z,2025-01-26T23:30:00Z,2600002565617-22L4294637
3,0.838,2025-01-26T22:30:00Z,2025-01-26T23:00:00Z,2600002565617-22L4294637
4,0.426,2025-01-26T22:00:00Z,2025-01-26T22:30:00Z,2600002565617-22L4294637


In [16]:
# This section looks up the price for each row in consumption
merged_kWh['interval_start'] = pd.to_datetime(merged_kWh['interval_start'])
merged_kWh['interval_end'] = pd.to_datetime(merged_kWh['interval_end'])
merged_tariffs['valid_from'] = pd.to_datetime(merged_tariffs['valid_from'])
merged_tariffs['valid_to'] = pd.to_datetime(merged_tariffs['valid_to'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13583 entries, 0 to 13582
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   consumption     13583 non-null  float64
 1   interval_start  13583 non-null  object 
 2   interval_end    13583 non-null  object 
 3   source_file     13583 non-null  object 
dtypes: float64(1), object(3)
memory usage: 424.6+ KB


In [ ]:


def get_rate(row):
    interval_start = row['interval_start']
    for _, price_row in price_df.iterrows():
        valid_from = price_row['valid_from']
        valid_to = price_row['valid_to']

        if pd.isna(valid_to):  # Use pd.isna() to reliably check for NaT (Not a Time)
            if interval_start >= valid_from:
                return price_row['value_inc_vat']
        elif valid_from <= interval_start < valid_to:
            return price_row['value_inc_vat']
    return None  # Return None if no matching rate is found


price_df = merged_tariffs[merged_tariffs['source_file']
                          == 'E-1R-INTELLI-VAR-22-10-14-H']
power_df = merged_kWh.copy()

power_df['rate'] = power_df.apply(get_rate, axis=1)

TypeError: '<=' not supported between instances of 'Timestamp' and 'str'

### Export price merge with main

In [37]:

def get_rate_sale(row):
    interval_start = row['interval_start']
    for _, price_row in export_df.iterrows():
        valid_from = price_row['valid_from']
        valid_to = price_row['valid_to']

        if pd.isna(valid_to):  # Use pd.isna() to reliably check for NaT (Not a Time)
            if interval_start >= valid_from:
                return price_row['value_inc_vat']
        elif valid_from <= interval_start < valid_to:
            return price_row['value_inc_vat']
    return None  # Return None if no matching rate is found


export_df = pd.read_csv('export.csv', parse_dates=True)
export_df['valid_from'] = pd.to_datetime(export_df['valid_from'], utc=True)
export_df['valid_to'] = pd.to_datetime(export_df['valid_to'], utc=True)
power_df['interval_start'] = pd.to_datetime(
    power_df['interval_start'], utc=True)
power_df['interval_end'] = pd.to_datetime(power_df['interval_end'], utc=True)
power_df['rate_sale'] = power_df.apply(get_rate_sale, axis=1)

C:\Users\Eshwar\AppData\Local\Temp\ipykernel_2852\10181034.py:20: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  power_df['interval_end'] = pd.to_datetime(power_df['interval_end'])


In [ ]:
power_df = pd.read_csv('power.csv', parse_dates=True)
export_df = pd.read_csv('export.csv', parse_dates=True)
export_df['valid_from'] = pd.to_datetime(export_df['valid_from'], utc=True)
export_df['valid_to'] = pd.to_datetime(export_df['valid_to'], utc=True)
power_df['interval_start'] = pd.to_datetime(
    power_df['interval_start'], utc=True)
power_df['interval_end'] = pd.to_datetime(power_df['interval_end'], utc=True)
# Ensure 'valid_from' is the index in export_df for efficient lookup
export_df = export_df.sort_values('valid_from').set_index('valid_from')

# Use merge_asof to find the matching rate for each interval_start
power_df = pd.merge_asof(
    power_df.sort_values('interval_start'),  # Sort power_df for merge_asof
    # Select only necessary column from export_df
    export_df[['value_inc_vat']],
    left_on='interval_start',
    # Merge on the index of export_df (valid_from)
    right_index=True,
    # Find the latest valid_from <= interval_start
    direction='backward',
    # You can use a large time delta to match any valid_from if valid_to is missing
    tolerance=pd.Timedelta('100000D'),
    allow_exact_matches=True                # Allow exact matches on 'valid_from'
).rename(columns={'value_inc_vat': 'rate_sale'})

## Final Data output

In [39]:
# power_df.drop(columns=['source_file'], inplace=True)
power_df.to_csv('historic_data.csv', index=False)